## Group Assignment

#### Import the necessary libraries

In [6]:
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from scipy.optimize import minimize

#### All questions in Parts I and II apply to a random sample of 15 stocks that your group will be assigned by running the following code.

#### Random Sample Selection 

In [7]:
np.random.seed (2051 + 6)
ticker_list = ['AAPL', 'AXP', 'BA', 'C','CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WMT', 'XOM']

stock_list = np.random.choice(ticker_list,15,replace=False)
print(f'These are the fifteen stocks assigned to you: {" ".join(stock_list)}')

These are the fifteen stocks assigned to you: DIS PG V UNH CSCO PFE JPM AXP JNJ INTC KO CAT WMT CVX XOM


### PART I

#### 1. Find the optimal portfolio over the period January 2015 - December 2019, using the fifteen stocks assigned to your group. Assume there are no short-selling constraints.

In [12]:
start = dt.datetime(2015,1,1)
end = dt.datetime(2019,12,31)

ticker_list= ['DIS', 'PG', 'V', 'UNH', 'CSCO', 'PFE', 'JPM', 'AXP', 'JNJ', 'INTC', 'KO', 'CAT', 'WMT', 'CVX', 'XOM']



In [13]:
returns = yf.download(ticker_list,start-pd.offsets.BDay(1),end)['Adj Close'].pct_change().dropna()

[*********************100%***********************]  15 of 15 completed


In [14]:
returns.head()

,AXP,CAT,CSCO,CVX,DIS,INTC,JNJ,JPM,KO,PFE,PG,UNH,V,WMT,XOM
Date,,,,,,,,,,,,,,,
2015-01-02,-0.000215,0.003824,-0.000723,0.003566,-0.004672,0.001929,-0.000478,0.004986,-0.001895,0.005779,-0.007135,-0.003067,0.010755,0.000232,0.004110
2015-01-05,-0.026446,-0.052786,-0.019920,-0.039972,-0.014613,-0.011276,-0.006984,-0.031045,0.000000,-0.005426,-0.004755,-0.016471,-0.022074,-0.002910,-0.027362
2015-01-06,-0.021312,-0.006435,-0.000370,-0.000462,-0.005304,-0.018637,-0.004914,-0.025929,0.007594,0.008344,-0.004555,-0.002018,-0.006444,0.007706,-0.005316
2015-01-07,0.021840,0.015497,0.009242,-0.000833,0.010230,0.020975,0.022076,0.001526,0.012482,0.013685,0.005245,0.010211,0.013398,0.026532,0.010133
2015-01-08,0.014175,0.010249,0.007693,0.022883,0.010341,0.018601,0.007863,0.022346,0.012095,0.020409,0.011435,0.047733,0.013412,0.021106,0.016645


In [15]:
returns_mon = returns.resample(rule = 'M').apply(lambda x: x.add(1).prod().sub(1))
returns_mon.tail()

,AXP,CAT,CSCO,CVX,DIS,INTC,JNJ,JPM,KO,PFE,PG,UNH,V,WMT,XOM
Date,,,,,,,,,,,,,,,
2019-08-31,-0.032162,-0.096225,-0.155054,-0.033952,-0.040207,-0.055784,-0.006897,-0.052931,0.045791,-0.076143,0.018553,-0.060279,0.017308,0.040247,-0.067624
2019-09-30,-0.017363,0.061428,0.055544,0.007475,-0.050561,0.086901,0.007946,0.071273,-0.003670,0.010689,0.034517,-0.066902,-0.048722,0.038684,0.031104
2019-10-31,-0.004662,0.099542,-0.031171,-0.020742,-0.003069,0.097031,0.020560,0.069935,-0.000184,0.067910,0.007440,0.162801,0.039823,-0.011965,-0.043054
2019-11-30,0.024216,0.050290,-0.046306,0.018462,0.166718,0.032540,0.048490,0.054755,-0.011595,0.013611,-0.019677,0.107519,0.033317,0.015606,0.020447
2019-12-31,0.034799,0.019277,0.050320,0.023222,-0.045841,0.027046,0.056804,0.052141,0.035019,0.010125,0.019744,0.053988,0.017993,0.007100,0.019815


In [56]:
rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)


In [57]:
# Define the portfolio returns function
def port_ret(weights):
    port_ret = np.dot(returns_mon*12,weights).mean()
    return port_ret

# Define the portfolio standard deviation function
def port_std(weights):
    port_std = np.sqrt(np.dot(weights, np.dot(returns_mon.cov()*12, weights)))
    return port_std

# Define the excess portfolio returns function
def ex_port_ret(weights):
    ex_port_ret = (np.dot(returns_mon,weights) - rf['RF']).mean()*12
    return ex_port_ret

# Define the excess portfolio standard deviation function
def ex_port_std(weights):
    ex_port_std = (np.dot(returns_mon,weights) - rf['RF']).std()*np.sqrt(12)
    return ex_port_std

# Define the negative Sharpe Ratio function that we will minimize
def neg_SR(weights):
    SR = ex_port_ret(weights) / ex_port_std(weights)
    return (-1)*SR

In [23]:
# By convention of minimize function it should be a function that returns zero for conditions
constraints = ({'type':'eq','fun': lambda weights: np.sum(weights) - 1})

# Weights must be between 0 and 1
boundaries=[(0,1)]
bounds = tuple(boundaries * len(returns_mon.columns))

# Initial Guess (equally weighted)
init_guess = np.full(len(returns_mon.columns), 1/len(returns_mon.columns))

In [24]:
# Optimal Portfolio (OP)
optimal_port = minimize(neg_SR,init_guess,#bounds = bounds,
                        constraints=constraints)
optimal_port

     fun: -2.3198714741313973
     jac: array([-0.05631357, -0.05625206, -0.05705136, -0.05622023, -0.0565168 ,
       -0.05642939, -0.05635992, -0.05633405, -0.05606759, -0.05678248,
       -0.05644104, -0.05635372, -0.05691686, -0.05587357, -0.05629501])
 message: 'Optimization terminated successfully'
    nfev: 259
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([-4.87455844e-01,  2.96799878e-04,  2.08813244e-02,  2.15295027e-01,
       -1.97080510e-01,  5.81793342e-02,  2.45194042e-02,  5.18674086e-01,
       -1.17409375e-01, -2.95323871e-01,  3.23590968e-01,  5.93982296e-01,
        8.61698920e-01,  4.35989933e-02, -5.63447553e-01])

#### 2. What are the weights of the stocks in the optimal portfolio?

In [25]:
# Weights of the assets in the optimal portfolio
for ticker in ticker_list:
    print(ticker + "  "+ str(round(optimal_port.x[ticker_list.index(ticker)],4)),end="  ")

DIS  -0.4875  PG  0.0003  V  0.0209  UNH  0.2153  CSCO  -0.1971  PFE  0.0582  JPM  0.0245  AXP  0.5187  JNJ  -0.1174  INTC  -0.2953  KO  0.3236  CAT  0.594  WMT  0.8617  CVX  0.0436  XOM  -0.5634  

#### 3. What was the annualized average monthly return for the optimal portfolio?

In [35]:
optimal_weights = optimal_port.x
optimal_annual_return = port_ret(optimal_weights) * 12

In [40]:
annualized_return = port_ret(optimal_port.x) * 12
annualized_return

5.005726089781277

In [36]:
optimal_annual_return

5.005726089781277

#### 4. What was the annualized monthly standard deviation for the optimal portfolio?

In [38]:
optimal_std = port_std(optimal_weights) * np.sqrt(12)

optimal_std 

0.6080899637304208

#### 5. What was the Sharpe Ratio of the optimal portfolio?

In [26]:
# Maximum Sharpe Ratio
-optimal_port.fun

2.3198714741313973

### PART II

#### 1. Create a DataFrame named optimal_weights to store the weights from a portfolio optimization performed on a rolling basis. Specifically, the optimization should use a 60 month rolling window, and be performed every month. The first 60 months correspond to the sample period for data_initial (01/2015 - 12/2019) created in question 3. Hence, the first observation in the optimal_weights DataFrame should be the weights you listed above in Part I. The next 60 month period should be 02/2015 - 01/2020, followed by 03/2015 - 02/2020, and so on. The last 60 month period should be 03/2018 - 02/2023. Again, assume there are no short-selling constraints.

In [60]:
window_size= 60

optimal_weights =pd.DataFrame()

for i in range(0,len(returns_mon.index)-window_size+1):
    
    def ex_port_ret(weights):
        ex_port_ret = (np.dot(returns_mon.iloc[i:i+window_size],weights) - rf.iloc[i:i+window_size]['RF']).mean()*12
        return ex_port_ret

    # Define the excess portfolio standard deviation function
    def ex_port_std(weights):
        ex_port_std = (np.dot(returns_mon.iloc[i:i+window_size],weights) - rf.iloc[i:i+window_size]['RF']).std()*np.sqrt(12)
        return ex_port_std

    # Define the negative Sharpe Ratio function that we will minimize
    def neg_SR(weights):
        SR = ex_port_ret(weights) / ex_port_std(weights)
        return (-1)*SR

    # By convention of minimize function it should be a function
    # that returns zero for conditions
    constraints = ({'type':'eq','fun': lambda weights: np.sum(weights) - 1})

    # Weights must be between 0 and 1
    boundaries=[(0,1)]
    bounds = tuple(boundaries * len(returns_mon.columns))

    # Initial Guess (equally weighted)
    init_guess = np.full(len(returns_mon.columns),1/len(returns_mon.columns))
    
    optimal_port = minimize(neg_SR,init_guess,bounds = bounds,constraints=constraints)
    
    optimal_weights = pd.concat([optimal_weights,
                                pd.DataFrame(optimal_port.x.reshape(1,len(ticker_list)).round(4),
                                             columns=[ticker_list],
                                             index=[returns_mon.iloc[i:i+window_size].index[-1]])],
                                axis=0)
    
optimal_weights

,DIS,PG,V,UNH,CSCO,PFE,JPM,AXP,JNJ,INTC,KO,CAT,WMT,CVX,XOM
2019-12-31,0.0,0.0,0.0,0.0,0.0,0.008,0.0,0.0504,0.0,0.0,0.1864,0.3162,0.4164,0.0226,0.0


#### 2. Create a new DataFrame named port_returns to store the monthly returns over the sample period 01/2020 - 02/2023 for the following portfolios:
 1) A monthly rebalanced portfolio using the rolling optimal weights. Specifically, the portfolio return for 01/2020 should be based on the weights as of 12/2019 (the same values detailed in Part I and in the first row in the optimal_weights DataFrame), the portfolio return for 02/2020 should be based on the weights as of 01/2020 and so on. Label the portfolio (MRoll_Reb_OP, an acronym for Monthly Rolling Rebalanced Optimal Portfolio). 
 2) A monthly rebalanced portfolio using the optimal weights as of 12/2019. Label the portfolio (M_Reb_OP, an acronym for Monthly Rebalancing Optimal Portfolio).
 3) A monthly rebalanced equally-weighted portfolio. Label the portfolio (M_Reb_EW, an acronym for Monthly Rebalancing Equally-Weighted Portfolio).
 4) A buy and hold portfolio, initially allocated according to the optimal weights as of 12/2019. Label the portfolio (BH_OP, an acronym for Buy and Hold Optimal Portfolio).
 5) A buy and hold portfolio, initially allocated equally across stocks. Label the portfolio (BH_EW, an acronym for Buy and Hold Equally-Weighted Portfolio).

#### 3. Create a DataFrame port_stats (with a 3x5 shape) to store the annualized average monthly return, the annualized monthly standard deviation and the Sharpe ratio for all 5 portfolios.

#### 4. Compute the cumulative return series for all portfolios and plot them in the same graph. 

#### 5. What was the cumulative return for the best performing portfolio? What was the cumulative return for the worst performing portfolio?

### PART III - Examination of the Size Effect

Please download the file "crsp_fall22.csv" located in folder Group_Assignment. The file contains the following columns:
* PERMNO - Permanent number (unique identifer for the stock)
* DATE
* PRC - The closing price
* VOL - Trading volume (in hundreds)
* RET - The simple return
* SHROUT - The number of shares outstanding (in thousands)
* CFACPR - Cumulative factor to adjust the closing price
* CFACSHR - Cumulative factor to adjust shares outstanding

#### 1. Create a DataFrame named fin_data by reading in the columns DATE, PERMNO, RET, PRC and SHROUT from the file 'crsp_fall22.csv'. Set the DATE as the index. In addition, change each index value to the end of the month. For example, change '1926-05-30' to 1926-05-31', or '1987-01-30' to '1987-01-31'. In addition, create a new column (labeled MKTCAP) to store the market capitalization (defined as PRC * SHROUT). Subsequently, permanently remove the PRC and SHROUT columns.

#### 2. How many stocks are there in the sample? How many of these stocks are in the sample over the entire sample period?

#### 3. Every June, split the stocks into quintiles (five groups) based on their market capitalization. The header of a DataFrame named size_qt with the resulting split can be found below. 

In [101]:
size_qt.sort_index().head()

,PERMNO,MKTCAP_QT
DATE,,
1990-06-30,10294,3
1990-06-30,65496,3
1990-06-30,10905,2
1990-06-30,32037,1
1990-06-30,66288,5


#### 4. Create a new DataFrame named data by using the merge_asof() function to merge the DataFrames fin_data and size_qt. Specifically, merge each stock's June quintile allocation with the stock's monthly returns for the next 12 months starting the following July. Subsequently, set DATE as the index in the DataFrame.

#### 5. Create a new DataFrame called quintiles to store the average monthly return for each quintile. The header of the DataFrame can be found below:

In [105]:
quintiles.head()

,Q1_RET,Q2_RET,Q3_RET,Q4_RET,Q5_RET
DATE,,,,,
1990-07-31,-0.001078,-0.041629,-0.044858,-0.040050,-0.028895
1990-08-31,-0.088672,-0.111944,-0.134610,-0.132793,-0.110817
1990-09-30,-0.045567,-0.090744,-0.092048,-0.099917,-0.080715
1990-10-31,-0.071408,-0.064675,-0.069424,-0.060729,-0.039041
1990-11-30,-0.011988,0.002373,0.033031,0.070116,0.099023


#### 6. Create a bar plot of the average monthly return for all five market capitalization groups, across the entire sample.

#### 7. Create a line plot of the cumulative return series for all five market capitalization groups.